# Assignment 4 — KDD Case Study (Wine Quality Regression)

Dataset: [`codesignal/wine-quality`](https://huggingface.co/datasets/codesignal/wine-quality) — red split (1,599 rows, 11 numeric features, target: quality 0–10).

Goal: Predict wine `quality` using KDD stages: Selection → Preprocessing → Transformation → Data Mining → Evaluation/Interpretation.

Environment: Python 3.11 (`.venv`), scikit-learn, pandas, seaborn, SHAP.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

from datasets import load_dataset
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.decomposition import PCA

import shap

sns.set_theme(style="whitegrid")
FIG_DIR = Path("figures")
FIG_DIR.mkdir(exist_ok=True)
RNG = 42
pd.set_option("display.max_columns", 30)

## Selection
- Load red wine split; target = `quality` (ordinal/treated as regression).
- Rationale: small, clean numeric dataset suitable for transformation (scaling/PCA) and regression models.

In [ ]:
ds = load_dataset("codesignal/wine-quality", split="red")
df = ds.to_pandas()
df.shape, df.head()

In [ ]:
target = "quality"
X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RNG)
X_train.shape, X_test.shape

### Target distribution and correlations

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(y, bins=10, kde=True)
plt.title("Quality distribution (red wine)")
plt.tight_layout()
target_fig = FIG_DIR / "target_distribution.png"
plt.savefig(target_fig)
target_fig

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df.corr(), annot=False, cmap="coolwarm", center=0)
plt.title("Feature correlation matrix")
plt.tight_layout()
corr_fig = FIG_DIR / "correlation_matrix.png"
plt.savefig(corr_fig)
corr_fig

## Preprocessing
- Check missingness and descriptive stats.
- All numeric features → scale.

In [ ]:
missing = df.isna().sum().sum()
df.describe(), missing

## Transformation
- Standardize features.
- PCA (variance explained) to inspect structure.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=6, random_state=RNG)
pca.fit(X_scaled)
explained = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(6,4))
plt.plot(range(1, len(explained)+1), explained, marker='o')
plt.xlabel('Components')
plt.ylabel('Cumulative explained variance')
plt.title('PCA variance (standardized)')
plt.tight_layout()
pca_fig = FIG_DIR / "pca_variance.png"
plt.savefig(pca_fig)
pca_fig

## Data Mining
- Pipelines: scale → model.
- Models: Dummy, RandomForest, GradientBoosting.
- Tuning: RandomizedSearchCV on GradientBoosting.

In [ ]:
preprocessor = Pipeline([("scaler", StandardScaler())])

def eval_reg(model, name: str):
    pipe = Pipeline([("prep", preprocessor), ("model", model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    return {"name": name, "rmse": rmse, "mae": mae, "r2": r2, "pipeline": pipe}

results = []
results.append(eval_reg(DummyRegressor(strategy="median"), "Dummy"))
results.append(eval_reg(RandomForestRegressor(n_estimators=300, random_state=RNG, n_jobs=-1), "RandomForest"))
results.append(eval_reg(GradientBoostingRegressor(random_state=RNG), "GradientBoosting"))

pd.DataFrame([{k:v for k,v in r.items() if k!="pipeline"} for r in results])

In [ ]:
# Pick best and tune GradientBoosting
best = sorted(results, key=lambda d: d["rmse"])[0]
best_name = best["name"]
param_dist = {
    "model__n_estimators": [200, 300, 400],
    "model__learning_rate": [0.05, 0.075, 0.1],
    "model__max_depth": [2, 3, 4],
}
search = RandomizedSearchCV(
    Pipeline([("prep", preprocessor), ("model", GradientBoostingRegressor(random_state=RNG))]),
    param_distributions=param_dist,
    n_iter=8,
    cv=3,
    scoring="neg_root_mean_squared_error",
    random_state=RNG,
    n_jobs=-1,
)
search.fit(X_train, y_train)
{'best_params': search.best_params_, 'best_cv_rmse': -search.best_score_}

In [ ]:
# Refit tuned model and evaluate
tuned = search.best_estimator_
tuned.fit(X_train, y_train)
preds = tuned.predict(X_test)
rmse = mean_squared_error(y_test, preds, squared=False)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)
{"rmse": rmse, "mae": mae, "r2": r2}

In [ ]:
# Residual analysis
residuals = y_test - preds
plt.figure(figsize=(6,4))
sns.scatterplot(x=preds, y=residuals, alpha=0.4)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residual')
plt.title('Residuals vs Predicted (tuned GB)')
plt.tight_layout()
resid_fig = FIG_DIR / "residuals_vs_pred.png"
plt.savefig(resid_fig)
resid_fig

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(residuals, bins=30, kde=True)
plt.title('Residual distribution (tuned GB)')
plt.tight_layout()
resid_hist_fig = FIG_DIR / "residual_hist.png"
plt.savefig(resid_hist_fig)
resid_hist_fig

In [ ]:
# SHAP summary (sampled)
X_sample = X_test.sample(n=min(400, len(X_test)), random_state=RNG)
X_proc = tuned.named_steps["prep"].transform(X_sample)
feature_names = tuned.named_steps["prep"].get_feature_names_out()
explainer = shap.Explainer(tuned.named_steps["model"], X_proc, feature_names=feature_names)
shap_values = explainer(X_proc, check_additivity=False)
shap.plots.beeswarm(shap_values, max_display=12, show=False)
plt.tight_layout()
shap_fig = FIG_DIR / "shap_beeswarm.png"
plt.savefig(shap_fig, bbox_inches="tight")
shap_fig

## Evaluation & Interpretation
- Tuned GB is best; compare RMSE/MAE/R² vs baselines.
- Residuals mostly centered, slight skew; monitor high-error cases.
- SHAP: key drivers include alcohol, sulphates, volatile acidity, density.
- PCA shows ~80% variance captured by first ~5 components.